# Habitat mapping of Askania Nova reserve using Sentinel-2 data and Random Forest

This Notebook is meant to reproduce mapping (primarily natural) habitats over the Falz-Fein Biosphere Reserve 'Askania Nova' territory. See Prylutskyi & Shapoval (2024) .... for more details.

The overall workflow consists of several consecutive steps:

1. Set the Environment, define the research area and set parameters for satellite data;
2. Access and prepare Sentinel-2 Surface Reflectance optical data;
3. Calculate derived statistics;
4. Image segmentation using SNIC algorithm, calculate median values of each band for each segment;
5. Download segments as an ESRI Shapefile to manual assignment of habitat classes for training data;
6. Annotation of segments (polygons) using desktop GIS, prepare geojson with numerical class IDs;
7. Upload annotated SNIC segments back to the Earth Engine to sample SNIC segments median image;
8. Stratified splitting of sampled data into training and testing subsets;
9. Train Random Forest classifier, and assess classification accuracy using testing subsets;
10. Classify the image of median values of bands for SNIC segments;
11. Export results as *.tiff and *.shp to Google Drive for visual inspection and preparation of publication-quality habitat map.

The remote sensing data used is based on Sentinel-2 Surface Reflectance, available through [Google Earth Engine data catalog](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED). Based on the optical bands, we calculate several statistics:

1. Surface reflectance of bands 'B2', 'B3', 'B4', 'B8', 'B11', 'B12' Sentinel-2 medianized per band across intervals of March-October (vegetation season) for 2015-2021 years;
2. NDVI and NDWI spectral indices medianized across 2015-2021 years for each month from March to October to control phenological signal.

As a result, we obtain a 22-band image for further analysis.

In the next step, we apply Superpixel clustering based on Simple Non-Iterative Clustering ([SNIC](https://developers.google.com/earth-engine/apidocs/ee-algorithms-image-segmentation-snic)) to our image in order to obtain homogenous groups of pixel likely represent homogenous habitats. We also need to extract the mean bands (cluster averages) from SNIC segmentation, as those data will play as predictors in further supervised classification of segments.

To produce training data, we first export vectorized SNIC segmentation to Google Drive, download it, and manually annotate segments with known habitat types in a desktop GIS. After that, `1_shp2geojson.R` script should be run to prepare data to upload back to Earth Engine.

As a Supervised classification method, we use a Random Forest with 50 trees and other parameters set to default. Training and testing points are obtained through the stratified split of data sampled from median band values of SNIC segments.

First, load the necessary libraries and initialize Google Earth Engine API. Authentification might be challenging and depends on how you run the Notebook. Whether it is Google Colab or a local execution via Jupyter Notebook/stand-alone code editor (I used [Positron](https://positron.posit.co/) and also tested in VS Code and Jupyter Lab), it's advisable for the first launch to set Google Chrome as a default web browser and make sure you are logging in to the same Google Account you use for Earth Engine. For me, an authentification went smoothly only in this way. You don't need to repeat manual authentication each time as earthengine-api credentials will be stored locally. However, it is going to require re-authorization from time to time.

In [ ]:
# Install required packages
# !pip install geemap geopandas earthengine-api

import ee
import geemap
import pandas as pd
import geopandas as gpd
import csv
import os

# Authenticate and initialise Earth Engine
ee.Authenticate()
ee.Initialize(project = 'ee-olegpril12') # Change 'ee-olegpril12' to your GEE project title

Now define the Areas of Interest. Read polygon borders of the Falz-Fein Biosphere Reserve "Askania Nova" from a geojson file and display it on the map to check validity.

In [3]:
# Define outer borders
geojson_path = './area_of_interest/bzan_borders.geojson'  # Path to GeoJSON

# Read and convert the GeoJSON to an Earth Engine feature collection
gdf = gpd.read_file(geojson_path)
aoi_borders = geemap.geojson_to_ee(gdf.__geo_interface__)

# map = geemap.Map()
# map.centerObject(aoi_borders, 11)
# map.addLayer(aoi_borders, {'color': 'red'}, 'Outer borders')
# map

Define priors for satellite data -- date range, optical bands, cloud cover percentage.

Set functions for cloud/cloud shadows masking for Sentinel-2 Surface Reflectance data. Access Image Collection, and combine optical data with vegetation indices.

In [5]:
# Parameters
start_year = 2015
end_year = 2021
start_month = 3  # March
end_month = 10   # October
cloud_percentage = 10 # per cent of cloud cover for upper threshold
bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']  # Optical bands of interest

# Function to mask clouds using QA60 band
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = (
        qa.bitwiseAnd(cloud_bit_mask).eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )
    return image.updateMask(mask).divide(10000).copyProperties(image, ["system:time_start"])

# Filter images by bounds and timestamp
s2_sr = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(aoi_borders)
    .filter(ee.Filter.calendarRange(start_year, end_year, 'year'))
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage))
    .filter(ee.Filter.notNull(['system:time_start']))  # Filter only images with timestamps
    .map(mask_s2_clouds)
    .select(bands)
)

# Calculate median composite for each band
median_composite = s2_sr.median()

# Function to calculate NDVI or NDWI for a given month
def calculate_index_for_month(month, index_name, bands):
    filtered = s2_sr.filter(ee.Filter.calendarRange(month, month, 'month'))
    index = filtered.map(
        lambda image: image.normalizedDifference(bands).rename(f"{index_name}_{month}")
    )
    return index.median()

# Generate NDVI and NDWI bands for each month in the range
ndvi_bands = []
ndwi_bands = []
for month in range(start_month, end_month + 1):
    ndvi_image = calculate_index_for_month(month, "NDVI", ['B8', 'B4'])
    ndvi_bands.append(ndvi_image)
    ndwi_image = calculate_index_for_month(month, "NDWI", ['B8', 'B3'])
    ndwi_bands.append(ndwi_image)

# Combine median composite, NDVI, and NDWI images for all months
final_image = median_composite
for ndvi_image, ndwi_image in zip(ndvi_bands, ndwi_bands):
    final_image = final_image.addBands(ndvi_image).addBands(ndwi_image)

# Get band names to verify
band_names = final_image.bandNames().getInfo()
print("Final Image Band Names:", band_names)

# Visualise the image on the map
visualization_params = {
    'min': 0.0,
    'max': 1.0,
    'bands': ['NDVI_4', 'NDVI_5', 'NDVI_9'], 
}

Map = geemap.Map()
Map.centerObject(aoi_borders, 10)
Map.addLayer(final_image, visualization_params, 'Median Image with NDVI')
Map.addLayer(aoi_borders, {'color': 'red'}, 'Outer borders')
Map

Final Image Band Names: ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'NDVI_3', 'NDWI_3', 'NDVI_4', 'NDWI_4', 'NDVI_5', 'NDWI_5', 'NDVI_6', 'NDWI_6', 'NDVI_7', 'NDWI_7', 'NDVI_8', 'NDWI_8', 'NDVI_9', 'NDWI_9', 'NDVI_10', 'NDWI_10']


Map(center=[46.47146654341297, 33.948553692283426], controls=(WidgetControl(options=['position', 'transparent_…

To visually examine the image and ensure reference for further manual annotation, we can download selected bands to Google Drive.

In [ ]:
# Select the desired bands
selected_bands = ['B2', 'B3', 'B4', 'B8', 'B11']
image_to_export = final_image.select(selected_bands).clip(aoi_borders)

# Export the image to local drive
out_file = 'sentinel2_2015_2021.tif'
task = ee.batch.Export.image.toDrive(
    image = image_to_export,
    description = 'Sentinel2MedianExport',
    folder = 'GEE_data',
    fileNamePrefix = out_file.replace('.tif', ''),
    region = aoi_borders.geometry().bounds().getInfo()['coordinates'],
    scale = 10,  # Sentinel-2 spatial resolution is 10m for selected bands
    maxPixels = 1e13
)
task.start()

# Monitor the export
print('Exporting... Check progress at: https://code.earthengine.google.com/tasks')

## Image segmentation and object-oriented classification

Superpixel clustering based on SNIC (Simple Non-Iterative Clustering). Outputs a band of cluster IDs and the per-cluster averages for each of the input bands. See [documentation](https://developers.google.com/earth-engine/apidocs/ee-algorithms-image-segmentation-snic) for more details.

In [6]:
# Clip the image to the Reserve's borders
image = final_image.clip(aoi_borders)

# Smaller. more homogenous segments
snic = ee.Algorithms.Image.Segmentation.SNIC(
    image = image,
    size = 10,          # Smaller size for finer segmentation
    compactness = 0.05, # Lower compactness for more homogeneous segments
    connectivity = 8    # Maintain 8-connectivity for smooth results
)

# Extract the mean bands (cluster averages) from SNIC segmentation
snic_means = snic.select(
    [f"{band}_mean" for band in image.bandNames().getInfo()]  # Extract mean band names
)

# Visualise the results
Map = geemap.Map()
Map.centerObject(aoi_borders, 12)  # Adjust zoom level for your AOI
Map.addLayer(
    image, 
    {"bands": ["B4", "B3", "B2"], "min": 0, "max": 0.3}, 
    "Input Image")
Map.addLayer(snic.randomVisualizer(), {}, "SNIC Segmentation")

# Display the map
Map

Map(center=[46.47146654341297, 33.948553692283426], controls=(WidgetControl(options=['position', 'transparent_…

In order to make training data for subsequent supervised classification, we need to download SNIC segments to the local drive, open it in desktop GIS (e.g., [QGIS](https://qgis.org/)), and manually assign habitat attribute to some of them, based on available ground truth data and expert knowledge. After that, we should save annotated SNIC segments into the new Shapefile (./snic_segments_annotated/snic_segments_annotated.shp) and run the `1_shp2geojson.R` script to prepare data to upload back to the Earth Engine. As a result, we must have a snic_segments_annotated.geojson file containing only segments annotated with numerical class labels (variable "class_id") stored in the "./snic_segments_annotated" directory.

In [ ]:
# Convert SNIC segments into vectors 
snic_vectors = snic.select("clusters").reduceToVectors(
    **{
        "geometryType": "polygon",
        "scale": 10,  # Scale in metres
        "maxPixels": 1e13,  # Increase if needed for large areas
        "geometry": aoi_borders.geometry(),  # Restrict vectors to your AOI
    }
)

# Export SNIC vectors to a shapefile
# Define the export task to Google Drive
export_task = ee.batch.Export.table.toDrive(
    collection = snic_vectors,
    description = "snic_segments",  # Name of the task
    folder = "EarthEngineExports",            # Optional: Folder in Google Drive
    fileFormat = "SHP"                        # Export format (Shapefile)
)

# Start the export task
export_task.start()

print("Task started. Monitor the task at: https://code.earthengine.google.com/tasks")

## Supervised classification of SNIC segments using Random Forest

We use annotated SNIC segments as polygon ground truth data at this step. Those data will be used to sample SNIC images, then split in a stratified way to ensure all classes are present in both training and testing datasets, and finally used for training the Random Forest classifier and assessing classification accuracy.

I strongly recommend to apply `1_shp2geojson.R` script to convert ESRI Shapefile into a geojson instead of exporting it as geojson using desktop GIS. This ensures geometry validity and assigns a numerical 'class_id' variable to each annotated segment. The latter is required since the Random Forest classification in Earth Engine is only possible with numerical class values.

In [7]:
# Load the GeoJSON file containing manually annotated segments
geojson_path = './snic_segments_annotated/snic_segments_annotated.geojson'
gdf = gpd.read_file(geojson_path)

# Check for missing 'class_num' property
missing_class_id = gdf[gdf['class_id'].isna()]
print(f"Number of features missing 'class_id': {len(missing_class_id)}")

if not missing_class_id.empty:
    print(missing_class_id)

Number of features missing 'class_id': 0


Then, the annotated segments are converted into an Earth Engine Feature Collection, and the SNIC segments are sampled to obtain point training data.

In [8]:
# Load the annotated GeoJSON file as a FeatureCollection
snic_segments_annotated = geemap.geojson_to_ee(gdf.__geo_interface__)

# Define feature properties for supervised classification
class_property = 'class_id'  # Numeric code for habitat classes

# Sample data from the SNIC means and annotate with class labels
sampled_data = snic_means.sampleRegions(
    **{
        "collection": snic_segments_annotated,
        "properties": [class_property],
        "scale": 10,  # Match scale of segmentation
    }
)

### Stratified split

By default, Eart Engine allows only a random split of the entire dataset at once based on a new property containing a [column of deterministic pseudorandom numbers](https://developers.google.com/earth-engine/apidocs/ee-featurecollection-randomcolumn?hl=en) added to a Feature Collection. As we can expect a pronounced class imbalance (different habitat types shall not be expected to occupy equal areas), we must apply a stratified split to ensure all classes appear in both training and testing datasets in the right proportion. To do so, we iteratively apply a randomColumn() function to each habitat type subset of sampled data, then merge them and split them by the value in a 'random' property.

In [9]:
# Get unique class IDs from the 'class_id' property and sort them
class_ids = sorted(
    snic_segments_annotated
    .distinct('class_id')  # Get unique class_id values
    .aggregate_array('class_id')  # Convert to an array
    .getInfo()  # Get the result as a Python list
)

# Function to create subsets with a random column
def create_random_subset(sampled_data, class_id):
    """
    Filters the FeatureCollection by class_id and adds a random column.
    
    Parameters:
    sampled_data (ee.FeatureCollection): The input FeatureCollection.
    class_id (int): The class_id to filter by.
    
    Returns:
    ee.FeatureCollection: The filtered FeatureCollection with a random column.
    """
    subset = sampled_data.filter(ee.Filter.eq('class_id', class_id))
    subset_with_random = subset.randomColumn()
    return subset_with_random

# Process each class_id independently and store results in a list
subsets = []
for class_id in class_ids:
    subset = create_random_subset(sampled_data, class_id)
    subsets.append(subset)

# Merge all subsets into one FeatureCollection
merged_fc = ee.FeatureCollection(subsets).flatten()

# Split the data into training (60%) and testing (40%)
split = 0.6
training_fc = merged_fc.filter(ee.Filter.lt('random', split))
testing_fc = merged_fc.filter(ee.Filter.gte('random', split))

Due to the Earth Engine's limitations, assessing and printing sizes of large Feature Collections might be extremely time-consuming. Thus, we can alternatively take a look at the first features of both splits to check their structure.

In [10]:
# # Print the sizes of the splits
# print("Training size:", training_fc.size().getInfo())
# print("Testing size:", testing_fc.size().getInfo())

print("Training data preview: ", training_fc.first().getInfo())
print("Testing data preview: ", testing_fc.first().getInfo())

Training data preview:  {'type': 'Feature', 'geometry': None, 'id': '0_326_0', 'properties': {'B11_mean': 0.22084000706672668, 'B12_mean': 0.15764915943145752, 'B2_mean': 0.04814097285270691, 'B3_mean': 0.07108181715011597, 'B4_mean': 0.06846360862255096, 'B8_mean': 0.1573646515607834, 'NDVI_10_mean': 0.32850342988967896, 'NDVI_3_mean': 0.3635842502117157, 'NDVI_4_mean': 0.4189620912075043, 'NDVI_5_mean': 0.4549817144870758, 'NDVI_6_mean': 0.35895782709121704, 'NDVI_7_mean': 0.2976934015750885, 'NDVI_8_mean': 0.34618261456489563, 'NDVI_9_mean': 0.30415549874305725, 'NDWI_10_mean': 0.33459722995758057, 'NDWI_3_mean': 0.3827904462814331, 'NDWI_4_mean': 0.3952062726020813, 'NDWI_5_mean': 0.41949212551116943, 'NDWI_6_mean': 0.3595248758792877, 'NDWI_7_mean': 0.2917270362377167, 'NDWI_8_mean': 0.33239802718162537, 'NDWI_9_mean': 0.32380470633506775, 'class_id': 1, 'random': 0.29864507510115956}}
Testing data preview:  {'type': 'Feature', 'geometry': None, 'id': '0_326_4', 'properties': {'B1

### Training a classifier

Train the Random Forest classfier.

In [11]:
# Train a Random Forest classifier
rf_classifier = ee.Classifier.smileRandomForest(50).train(
    features = training_fc,
    classProperty = class_property,
    inputProperties = snic_means.bandNames().getInfo()
)

### Accuracy assessment

Now we calculate and export the main accuracy metrics as CSV files: overall accuracy, Cohen's Kappa, and confusion matrix. We do it to test datasets and evaluate the quality of prediction.

While Earth Engine has an in-built function errorMatrix(), its execution and export are extremely time-consuming. So we apply a workaround as making a prediction over the testing datasets and exporting it to the ./accuracy/classified_test_data.csv, then calculating accuracy metrics using sklearn.metrics Python package, locally.

In [10]:
# Evaluate the classifier with testing data
testing_classified = testing_fc.classify(rf_classifier)

# Export the image sample feature collection to Drive as a CSV file.
task = ee.batch.Export.table.toDrive(
    collection = testing_classified.select(['class_id', 'classification']),
    description = 'classified_test_data',
    folder = "EarthEngineExports",
    fileFormat = 'CSV',
)
task.start()

print('Exporting... Check progress at: https://code.earthengine.google.com/tasks')

Exporting... Check progress at: https://code.earthengine.google.com/tasks


In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score

# Load the CSV file
data = pd.read_csv("./accuracy/classified_test_data.csv")

# Extract the ground truth and predicted classes
y_true = data['class_id']  # Reference class (ground truth)
y_pred = data['classification']  # Predicted class

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

# Compute additional accuracy metrics
overall_accuracy = accuracy_score(y_true, y_pred)
kappa = cohen_kappa_score(y_true, y_pred)

print(f"Overall Accuracy: {overall_accuracy}")
print(f"Kappa Coefficient: {kappa}")

# Export confusion matrix to CSV
cm_df = pd.DataFrame(cm, index=sorted(data['class_id'].unique()), columns=sorted(data['classification'].unique()))
cm_df.to_csv("./accuracy/test_confusion_matrix.csv", index_label="Class ID")
print("Confusion matrix saved as confusion_matrix.csv")

# Save overall_accuracy to CSV
overall_accuracy_df = pd.DataFrame({'Metric': ['Overall Accuracy'], 'Value': [overall_accuracy]})
overall_accuracy_df.to_csv("./accuracy/test_overall_accuracy.csv", index=False)
print("Overall accuracy saved as test_overall_accuracy.csv")

# Save kappa to CSV
kappa_df = pd.DataFrame({'Metric': ['Kappa Coefficient'], 'Value': [kappa]})
kappa_df.to_csv("./accuracy/test_kappa.csv", index=False)
print("Kappa coefficient saved as test_kappa.csv")

Confusion Matrix:
[[  711     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0]
 [    0  1371     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0 11462     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0   259     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0  5465     0     0     0     1     0     0     0     3     0     0     0     0     0     0]
 [    0     0     0     0     0  3181     0     0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0   829     0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0   967     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0   

### Classification of SNIC segments

The final step is the classification of the SNIC segmentation using our trained classifier.

In [12]:
# Apply the classifier to classify all SNIC segments
classified_segments = snic_means.classify(rf_classifier).clip(aoi_borders)

# Import necessary libraries
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors  # Import the colors module
import numpy as np

# Get the unique class IDs from the feature collection
unique_classes = snic_segments_annotated.aggregate_array('class_id').distinct().getInfo()
unique_classes.sort()  # Sort the class IDs for consistency

# Generate a dynamic colour palette
num_classes = len(unique_classes)
palette = plt.cm.get_cmap('tab10', num_classes)  # Use a colormap with enough distinct colours
colour_palette = [mcolors.rgb2hex(palette(i)) for i in range(num_classes)]

# Define visualisation parameters
vis_params = {
    "min": min(unique_classes),
    "max": max(unique_classes),
    "palette": colour_palette,
}

# Visualise the classification results
Map = geemap.Map()
Map.centerObject(aoi_borders, 12)
Map.addLayer(image, {"bands": ["B4", "B3", "B2"], "min": 0, "max": 0.3}, "Input Image")
Map.addLayer(snic.randomVisualizer(), {}, "SNIC Segmentation")
Map.addLayer(
    classified_segments,
    vis_params,
    "Classified Segments"
)
Map.addLayer(aoi_borders, {"color": "white"}, "AOI Borders", False)

# Display the map
Map

Now, depending on our needs, we can export classified segmentation into a .tiff and/or .shp file. If one prefers a Shapefile to manual inspection/post-processing, I advise using the `2_postprocessing.R` script afterwards. It contains the pipeline for the re-assignment of text labels, enables visual settings and exports the classification into a publication-ready map. It also processes accuracy metrics to reconcile a unified accuracy report.

Export classified SNIC segments into a geotiff raster file.

In [ ]:
# Define GeoTIFF export task
geotiff_task = ee.batch.Export.image.toDrive(
    **{
        "image": classified_segments,
        "description": "SNIC_Classification",
        "folder": "EarthEngineExports",
        "fileNamePrefix": "classified_segments",
        "scale": 10,  # Match segmentation scale
        "region": aoi_borders.geometry().bounds(),
        "maxPixels": 1e13,
    }
)
geotiff_task.start()

Export classified SNIC segments into ESRI Shapefile.

In [13]:
# Convert classified segments to vectors
classified_vectors = classified_segments.reduceToVectors(
    **{
        "geometryType": "polygon",
        "scale": 10,
        "maxPixels": 1e13,
        "geometry": aoi_borders.geometry(),
    }
)

# Define the export task to Google Drive
export_task = ee.batch.Export.table.toDrive(
    collection = classified_vectors,
    description = "ExportClassifiedSegments", # Name of the task
    folder = "EarthEngineExports",            # Optional: Folder in Google Drive
    fileFormat = "SHP"                        # Export format (Shapefile)
)

# Start the export task
export_task.start()

print('Exporting... Check progress at: https://code.earthengine.google.com/tasks')

Exporting... Check progress at: https://code.earthengine.google.com/tasks


### Pixel-based classification

We can also classify the original image.

In [17]:
# Define the new band names
new_band_names = [
    'B2_mean', 'B3_mean', 'B4_mean', 'B8_mean', 'B11_mean', 'B12_mean', 
    'NDVI_3_mean', 'NDWI_3_mean', 'NDVI_4_mean', 'NDWI_4_mean', 
    'NDVI_5_mean', 'NDWI_5_mean', 'NDVI_6_mean', 'NDWI_6_mean', 
    'NDVI_7_mean', 'NDWI_7_mean', 'NDVI_8_mean', 'NDWI_8_mean', 
    'NDVI_9_mean', 'NDWI_9_mean', 'NDVI_10_mean', 'NDWI_10_mean'
]

# Apply the classifier to classify all SNIC segments
classified_pixels = final_image.rename(new_band_names).classify(rf_classifier).clip(aoi_borders)

# Import necessary libraries
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors  # Import the colors module
import numpy as np

# Get the unique class IDs from the feature collection
unique_classes = snic_segments_annotated.aggregate_array('class_id').distinct().getInfo()
unique_classes.sort()  # Sort the class IDs for consistency

# Generate a dynamic colour palette
num_classes = len(unique_classes)
palette = plt.cm.get_cmap('tab10', num_classes)  # Use a colormap with enough distinct colours
colour_palette = [mcolors.rgb2hex(palette(i)) for i in range(num_classes)]

# Define visualisation parameters
vis_params = {
    "min": min(unique_classes),
    "max": max(unique_classes),
    "palette": colour_palette,
}

# Visualise the classification results
Map = geemap.Map()
Map.centerObject(aoi_borders, 12)
Map.addLayer(image, {"bands": ["B4", "B3", "B2"], "min": 0, "max": 0.3}, "Input Image")
# Map.addLayer(snic.randomVisualizer(), {}, "SNIC Segmentation")
Map.addLayer(
    classified_pixels,
    vis_params,
    "Classified pixel image"
)
Map.addLayer(aoi_borders, {"color": "white"}, "AOI Borders", False)

# Display the map
Map

Map(center=[46.47146654341297, 33.948553692283426], controls=(WidgetControl(options=['position', 'transparent_…

Export classified image to Google Drive

In [18]:
# Define GeoTIFF export task
geotiff_task = ee.batch.Export.image.toDrive(
    **{
        "image": classified_pixels,
        "description": "Pixel_Classification",
        "folder": "EarthEngineExports",
        "fileNamePrefix": "classified_pixels",
        "scale": 10,  # Match segmentation scale
        "region": aoi_borders.geometry().bounds(),
        "maxPixels": 1e13,
    }
)
geotiff_task.start()

print('Exporting... Check progress at: https://code.earthengine.google.com/tasks')

# End of the notebook